In [7]:
!pip install selenium
!pip install google-colab-selenium
!pip install openpyxl
!pip install webdriver-manager

In [8]:
import logging
logger = logging.getLogger("my_parser")
logger.setLevel(logging.DEBUG)

file_handler = logging.FileHandler("parser.log", mode='w', encoding='utf-8')
file_handler.setLevel(logging.DEBUG)

console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

logger.addHandler(file_handler)
logger.addHandler(console_handler)

In [9]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import glob
import os

In [10]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
chrome_options.add_argument("--ignore-certificate-errors")

In [19]:
cities = ['moskva', 'sankt-peterburg']
cities_2 = ['ekaterinburg', 'novosibirsk', 'krasnodar', 'krasnoyarsk', 'chelyabinsk', 'tyumen', 'ufa', 'rostov-na-donu']
cities_3 = ['kazan', 'perm', 'samara', 'nizhniy_novgorod', 'habarovsk', 'omsk', 'saratov', 'tomsk', 'voronezh', 'volgograd', 'irkutsk', 'sochi', 'surgut', 'yaroslavl', 'vladivostok']
cities_4 = ['kaluga','balashiha', 'podolsk', 'mytischi', 'pyatigorsk']

In [20]:
def get_content_page(html, collected_links):
    """Сбор данных с текущей страницы."""
    soup = BeautifulSoup(html, 'html.parser')
    blocks = soup.find_all('div', {"data-test-component": "ProductOrAdCard"})
    data_list = []

    for block in blocks:
        try:
            name = block.find('span', {'data-test-block': "ProductName"}).text.strip()
        except:
            name = 'нет названия'

        try:
            city = block.find('div', {'data-test-component': "Badges"}).text.split('%')[-1].strip()
        except:
            city = 'город не указан'

        try:
            price = block.find('p', {'data-test-block': "ProductPrice"}).text.replace('₽руб.', '').replace('\xa0', '').strip()
        except:
            price = 'нет цены'
        try:
             link = "https://youla.ru" + block.find('div').find('span').find('a').get('href')
        except:
            link = 'ссылка не найдена'

        if link not in collected_links and name != 'нет названия':
            collected_links.add(link)
            data_list.append({
                'name': name,
                'city': city,
                'price': price,
                'link': link
            })

    return data_list


def parser(url, data_list_count=1000):

    browser = webdriver.Chrome(options=chrome_options)

    try:
        browser.get(url)
        time.sleep(2)
        collected_links = set()
        data_list_pages = []
        last_count = 0

        while len(data_list_pages) < data_list_count:
            new_data = get_content_page(browser.page_source, collected_links)
            data_list_pages.extend(new_data)
            logger.debug(f'Собрано {len(data_list_pages)} позиций')


            browser.execute_script("window.scrollBy(0, 500);")
            # Ждем появления новых данных
            time.sleep(5)

            if last_count == len(data_list_pages):
                logger.info("Больше новых данных не найдено.")
                break
            last_count = len(data_list_pages)

        return data_list_pages[:data_list_count]

    except Exception as ex:
        logger.error(f'Ошибка: {ex}')
    finally:
        browser.quit()


def save_excel(data):
    """Сохранение данных в Excel."""
    dataframe = pd.DataFrame(data)
    dataframe.to_excel('data_youla_' + '.xlsx', index=False)
    logger.info('Данные сохранены в "data_youla_' + '.xlsx"')


if __name__ == "__main__":
        for i in cities:
          url = 'https://youla.ru/' + i +'/auto/s-probegom?attributes[auto_state][0]=161308&attributes[auto_gear_type][0]=20807&attributes[auto_gear_type][1]=20808'

          logger.info('Запуск парсера...')
          data = parser(url, 1000)
          save_excel(data)


Выходные данные были обрезаны до нескольких последних строк (5000).
ERROR:my_parser:Нет названия
2025-03-04 20:36:47,871 - my_parser - ERROR - Город не указан
2025-03-04 20:36:47,871 - my_parser - ERROR - Город не указан
2025-03-04 20:36:47,871 - my_parser - ERROR - Город не указан
ERROR:my_parser:Город не указан
2025-03-04 20:36:47,875 - my_parser - ERROR - Нет цены
2025-03-04 20:36:47,875 - my_parser - ERROR - Нет цены
2025-03-04 20:36:47,875 - my_parser - ERROR - Нет цены
ERROR:my_parser:Нет цены
2025-03-04 20:36:47,883 - my_parser - ERROR - Ссылка не найдена
2025-03-04 20:36:47,883 - my_parser - ERROR - Ссылка не найдена
2025-03-04 20:36:47,883 - my_parser - ERROR - Ссылка не найдена
ERROR:my_parser:Ссылка не найдена
2025-03-04 20:36:47,886 - my_parser - ERROR - Нет названия
2025-03-04 20:36:47,886 - my_parser - ERROR - Нет названия
2025-03-04 20:36:47,886 - my_parser - ERROR - Нет названия
ERROR:my_parser:Нет названия
2025-03-04 20:36:47,891 - my_parser - ERROR - Город не указан
2

KeyboardInterrupt: 

*  Москва - небитый - механика - https://youla.ru/moskva/auto/s-probegom?attributes[auto_state][0]=161308&attributes[auto_gear_type][0]=20809
*  Москва - небитый - автомат - https://youla.ru/moskva/auto/s-probegom?attributes[auto_state][0]=161308&attributes[auto_gear_type][0]=20810
*  Санкт-Петербург - небитый - механика - https://youla.ru/sankt-peterburg/auto/s-probegom?attributes[auto_state][0]=161308&attributes[auto_gear_type][0]=20809
*  Санкт-Петербург - небитый - автомат - https://youla.ru/sankt-peterburg/auto/s-probegom?attributes[auto_state][0]=161308&attributes[auto_gear_type][0]=20810
*  Екатеринбург - небитый - https://youla.ru/ekaterinburg/auto/s-probegom?attributes[auto_state][0]=161308

*  мск - небитый - вариатор + робот - https://youla.ru/moskva/auto/s-probegom?attributes[auto_state][0]=161308&attributes[auto_gear_type][0]=20807&attributes[auto_gear_type][1]=20808
*  спб - небитый - вариатор + робот - https://youla.ru/sankt-peterburg/auto/s-probegom?attributes[auto_state][0]=161308&attributes[auto_gear_type][0]=20807&attributes[auto_gear_type][1]=20808

In [ ]:
!unzip /content/data_youla.zip -d /content/unzipped_folder

Archive:  /content/data_youla.zip
   creating: /content/unzipped_folder/data_youla/
  inflating: /content/unzipped_folder/data_youla/data_yula_balashiha.xlsx  
  inflating: /content/unzipped_folder/data_youla/data_yula_chelyabinsk.xlsx  
  inflating: /content/unzipped_folder/data_youla/data_yula_habarovsk.xlsx  
  inflating: /content/unzipped_folder/data_youla/data_yula_irkutsk.xlsx  
  inflating: /content/unzipped_folder/data_youla/data_yula_kaluga.xlsx  
  inflating: /content/unzipped_folder/data_youla/data_yula_kazan.xlsx  
  inflating: /content/unzipped_folder/data_youla/data_yula_krasnodar.xlsx  
  inflating: /content/unzipped_folder/data_youla/data_yula_krasnoyarsk.xlsx  
  inflating: /content/unzipped_folder/data_youla/data_yula_moskva_вариатор+робот.xlsx  
  inflating: /content/unzipped_folder/data_youla/data_yula_mytischi.xlsx  
  inflating: /content/unzipped_folder/data_youla/data_yula_nizhniy_novgorod.xlsx  
  inflating: /content/unzipped_folder/data_youla/data_yula_omsk.xls

In [12]:
folder_path = "/content/unzipped_folder/data_youla"
file_pattern = os.path.join(folder_path, "data_yula_*.xlsx")

excel_files = glob.glob(file_pattern)
df_list = []

for file in excel_files:
    df = pd.read_excel(file)
    df_list.append(df)

df_merged = pd.concat(df_list, ignore_index=True)

output_file = os.path.join(folder_path, "merged_data_yula.xlsx")
df_merged.to_excel(output_file, index=False)



ValueError: No objects to concatenate

In [13]:
TARGET_FEATURES = {"Поколение", "Руль", "Кузов", "Цвет", "Мощность", "Тип двигателя", "Объем двигателя", "Коробка передач", "Привод", "Пробег", "Описание"}


In [14]:

def get_info(url):
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    wait = WebDriverWait(driver, 5)

    data = {'URL': url}

    try:


        more_params_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'sc-cOxWqc.sc-cfIUOQ.jcXBYC.efVfLm')))
        more_params_btn.click()
        time.sleep(1)



        labels = driver.find_elements(By.TAG_NAME, "dt")
        values = driver.find_elements(By.TAG_NAME, "dd")


        for label, value in zip(labels, values):
            key = label.text.strip()
            val = value.text.strip()
            if key in TARGET_FEATURES:
                data[key] = val


    except Exception as e:
        logger.error(f"Ошибка при парсинге {url}: {e}")
        data['Ошибка'] = str(e)

    finally:
        driver.quit()

    return data




In [16]:
get_info('https://youla.ru/vladivostok/auto/s-probegom/suzuki-alto-2010-67c592fc26bf6d41690e0dd4?source_view=catalog')

{'URL': 'https://youla.ru/vladivostok/auto/s-probegom/suzuki-alto-2010-67c592fc26bf6d41690e0dd4?source_view=catalog',
 'Описание': 'Продам машину, полностью на уверенном ходу \nГнили нет \nСалон чистый\nДокументы чистые\nСигналка с автозапуском \nМасло свежее',
 'Поколение': '5 поколение',
 'Руль': 'Правый',
 'Кузов': 'Хетчбэк',
 'Цвет': 'Белый',
 'Мощность': '54 л.с.',
 'Тип двигателя': 'Бензиновый',
 'Объем двигателя': '0.7 л',
 'Коробка передач': 'Автомат',
 'Привод': 'Передний',
 'Пробег': '180000 км'}

In [17]:
df_links = pd.read_excel("merged_data_yula.xlsx")


parsed_data = []

for index, row in df_links[1400:1700].iterrows():
    url = row['link']
    logger.debug(f"Обрабатываем {index + 1}/{len(df_links)}: {url}")
    parsed_data.append(get_info(url))

    time.sleep(2)

df_parsed = pd.DataFrame(parsed_data)

df_parsed.to_excel('data_youla.xlsx', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'merged_data_yula.xlsx'